In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import sys
sys.path.append("/content/drive/My Drive/GoogleColab/pytorch3d_packages")

In [9]:
# Imports
import os
import glob
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

# Paths
zip_path = "/content/drive/MyDrive/GoogleColab/ShapeNetCore.zip"
extract_path = "/content/ShapeNetCore/ShapeNetCore"

# Extract zip if not already extracted
if not os.path.exists(extract_path):
    print(f"Extracting {zip_path} to {extract_path} ...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall("/content/ShapeNetCore")
    print("Extraction complete.")

# ClassNames
SYNSET_TO_NAME = {
    "02808440": "bathtub",
    "03642806": "laptop",
    "02992529": "cellphone",
    "03211117": "display",
    "03046257": "clock"
}

# Verify extraction
top_level_ids = os.listdir(extract_path)
top_level_names = [SYNSET_TO_NAME.get(syn, syn) for syn in top_level_ids]
print("Top-level classes:", top_level_names)

# Dataset
class ShapeNetPointCloudDataset(Dataset):
    def __init__(self, root_dir, num_points=1024):
        self.root_dir = root_dir
        self.items = []
        self.classes = []
        self.num_points = num_points

        # Collect class dirs
        class_dirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        class_dirs.sort()
        self.classes = class_dirs
        self.class_to_idx = {c: i for i, c in enumerate(self.classes)}

        # Collect .obj files
        for c in self.classes:
            class_dir = os.path.join(root_dir, c)
            obj_files = glob.glob(os.path.join(class_dir, "**", "*.obj"), recursive=True)
            for f in obj_files:
                self.items.append((f, self.class_to_idx[c]))

        print(f"Detected {len(self.items)} meshes across {len(self.classes)} classes.")

    def __len__(self):
        return len(self.items)

    def __getitem__(self, idx):
        # Load vertices from .obj file
        filepath, label = self.items[idx]
        verts = self._load_vertices(filepath)
        if verts is None:
            # Skip invalid files
            return self.__getitem__((idx + 1) % len(self))
        pc = self._fix_pointcloud(verts) # Normalize and pad/crop
        return {"points": pc, "label": torch.tensor(label, dtype=torch.long)}

    def _load_vertices(self, filepath):
        # Read vertex coordinates from .obj file
        verts = []
        try:
            with open(filepath, "r") as f:
                for line in f:
                    if line.startswith("v "):
                        parts = line.strip().split()
                        verts.append([float(parts[1]), float(parts[2]), float(parts[3])])
        except Exception as e:
            print(f"[WARN] Skipping {filepath}: {e}")
            return None
        if len(verts) == 0:
            return None
        return torch.tensor(verts, dtype=torch.float32)

    def _fix_pointcloud(self, verts: torch.Tensor):
        N = self.num_points
        V = verts.shape[0]

        # Normalize (center + scale)
        verts = verts - verts.mean(dim=0, keepdim=True)
        scale = torch.max(torch.norm(verts, dim=1))
        if scale > 0:
            verts = verts / scale

         # Crop or pad points to fixed size
        if V == N:
            pc = verts
        elif V > N:
            idx = torch.randperm(V)[:N]
            pc = verts[idx]
        else:
            pad = torch.zeros((N - V, 3))
            pc = torch.cat([verts, pad], dim=0)
        return pc

# Collate
def collate_pointclouds(batch):
    points = torch.stack([b["points"] for b in batch], dim=0)  # (B, N, 3)
    labels = torch.stack([b["label"] for b in batch])
    return {"points": points, "labels": labels}

# PointNet Model
class PointNetClassifier(nn.Module):
    """
    Basic PointNet model: applies MLP to each point, then max-pools
    to get global features, followed by fully connected layers.
    """
    def __init__(self, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(3, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, 1024)
        self.fc4 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 256)
        self.fc6 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        # x: (B, N, 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.max(x, dim=1)[0]  # Global feature (B, 1024)
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.dropout(x)
        x = self.fc6(x)
        return x

# Training Loop
def train_model(zip_path=zip_path, extract_path=extract_path, num_points=1024,
                epochs=5, batch_size=4, lr=1e-3, device=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    dataset = ShapeNetPointCloudDataset(extract_path, num_points=num_points)
    if len(dataset) == 0:
        raise RuntimeError("Dataset is empty — check .obj files and folder structure!")

    class_names = [SYNSET_TO_NAME.get(c, c) for c in dataset.classes]

    # Train/val split
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                              collate_fn=collate_pointclouds)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                            collate_fn=collate_pointclouds)

    model = PointNetClassifier(num_classes=len(dataset.classes)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for batch in train_loader:
            points, labels = batch['points'].to(device), batch['labels'].to(device)
            optimizer.zero_grad()
            outputs = model(points)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        train_acc = 100. * correct / total

        model.eval()
        val_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for batch in val_loader:
                points, labels = batch['points'].to(device), batch['labels'].to(device)
                outputs = model(points)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * labels.size(0)
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        val_acc = 100. * correct / total
        print(f"Epoch {epoch+1}/{epochs} - ValLoss: {val_loss/len(val_loader.dataset):.4f}, ValAcc: {val_acc:.2f}%")

    return model

#  Run training classifying 3D figures
if __name__ == "__main__":
    model = train_model(epochs=5, batch_size=4, lr=1e-3)



Top-level classes: ['laptop', 'clock', 'bathtub', 'display', 'cellphone']
Detected 3891 meshes across 5 classes.
Epoch 1/5 - ValLoss: 0.4782, ValAcc: 84.34%
Epoch 2/5 - ValLoss: 0.3820, ValAcc: 88.19%
Epoch 3/5 - ValLoss: 0.3344, ValAcc: 88.58%
Epoch 4/5 - ValLoss: 0.3609, ValAcc: 88.32%
Epoch 5/5 - ValLoss: 0.4057, ValAcc: 87.42%
